In [2]:
import datetime
import pandas as pd
import requests as re
from pprint import pprint as pp
import threading
import time
import sqlite3

In [49]:
def getData(token, fromm, to, time_frame):
        global last_data,superdf
        data=re.get(f"https://api.kite.trade/instruments/historical/{token}/{time_frame}?from={fromm}&to={to}",headers=headers)
        df=pd.DataFrame(data.json()['data']['candles'])
        df.columns = ["Timestamp", "open", "high", "low","close","volume"]
        superdf=SuperTrend(df)
        superdf = superdf.iloc[-1]
        newrow = superdf.to_dict()
        print(type(newrow))
        return newrow

In [42]:
def SuperTrendddd(newrow,data, period = 10, multiplier=3, ohlc=['Open', 'High', 'Low', 'Close']):
        global supertrend
        tr0 = abs(newrow[2] - newrow[3])
        tr1 = abs(newrow[2] - data["close"])
        tr2 = abs(newrow[3]- data["close"])
        TR = round(max([tr0, tr1, tr2]),2)
        
        ST=data["ST"]
        ATR = (data['ATR'] * (period-1))+TR/period
        BUB = round(((newrow[2] + newrow[3]) / 2) + (multiplier * ATR),2)
        BLB = round(((newrow[2] + newrow[3]) / 2) - (multiplier * ATR),2)
        
        if (BUB<data["final_ub"])|(data["close"]>data["final_ub"]):
            final_ub= BUB
        else:
            final_ub=data["final_ub"]

        if (BLB>data["final_lb"])|(data["close"]<data["final_lb"]):
            final_lb=BLB
        else:
            final_lb=data["final_lb"]
            
            
       
        if (ST==data['final_ub']) & (newrow[4]<=final_ub):
            ST =final_ub
        elif (data["ST"]==data["final_ub"])&(newrow[4]>final_ub):
            ST=final_lb
        elif (data["ST"]==data["final_lb"])&(newrow[4]>=final_lb):
            ST=final_lb
        elif (data["ST"]==data["final_lb"])&(newrow[4]<final_lb):
            ST=final_ub
            
    
        if (ST<data["close"]) :
            STX="green"
        else:
            STX="red"
            
        newrow_list = {"Timestamp":newrow[0],"open":newrow[1],"high":newrow[2],"low":newrow[3],"close":newrow[4],"volume":newrow[5],"ATR":ATR,"final_ub":final_ub,"final_lb":final_lb,"ST":ST, "STX":STX}
        
        
        return newrow_list

In [43]:
def SuperTrend(data, period = 10, multiplier=3, ohlc=['open', 'high', 'low', 'close']):
        global supertrend
        data['tr0'] = abs(data["high"] - data["low"])
        data['tr1'] = abs(data["high"] - data["close"].shift(1))
        data['tr2'] = abs(data["low"]- data["close"].shift(1))
        data["TR"] = round(data[['tr0', 'tr1', 'tr2']].max(axis=1),2)
        data["ATR"]=0.00
        data['BUB']=0.00
        data["BLB"]=0.00
        data["final_ub"]=0.00
        data["final_lb"]=0.00
        data["ST"]=0.00
        for i, row in data.iterrows():
            if i == 0:
                data.loc[i,'ATR'] = 0.00#data['ATR'].iat[0]
            else:
                data.loc[i,'ATR'] = ((data.loc[i-1,'ATR'] * (period-1))+data.loc[i,'TR'])/period
        data['BUB'] = round(((data["high"] + data["low"]) / 2) + (multiplier * data["ATR"]),2)
        data['BLB'] = round(((data["high"] + data["low"]) / 2) - (multiplier * data["ATR"]),2)
        
        if (data.loc[i,"BUB"]<data.loc[i-1,"final_ub"])|(data.loc[i-1,"close"]>data.loc[i-1,"final_ub"]):
            data.loc[i,"final_ub"]=data.loc[i,"BUB"]
        else:
            data.loc[i,"final_ub"]=data.loc[i-1,"final_ub"]

        if (data.loc[i,"BLB"]>data.loc[i-1,"final_lb"])|(data.loc[i-1,"close"]<data.loc[i-1,"final_lb"]):
            data.loc[i,"final_lb"]=data.loc[i,"BLB"]
        else:
            data.loc[i,"final_lb"]=data.loc[i-1,"final_lb"]

        for i, row in data.iterrows():
            if i==0:
                data.loc[i,"ST"]=0.00
            elif (data.loc[i-1,"ST"]==data.loc[i-1,"final_ub"]) & (data.loc[i,"close"]<=data.loc[i,"final_ub"]):
                data.loc[i,"ST"]=data.loc[i,"final_ub"]
            elif (data.loc[i-1,"ST"]==data.loc[i-1,"final_ub"])&(data.loc[i,"close"]>data.loc[i,"final_ub"]):
                data.loc[i,"ST"]=data.loc[i,"final_lb"]
            elif (data.loc[i-1,"ST"]==data.loc[i-1,"final_lb"])&(data.loc[i,"close"]>=data.loc[i,"final_lb"]):
                data.loc[i,"ST"]=data.loc[i,"final_lb"]
            elif (data.loc[i-1,"ST"]==data.loc[i-1,"final_lb"])&(data.loc[i,"close"]<data.loc[i,"final_lb"]):
                data.loc[i,"ST"]=data.loc[i,"final_ub"]

        # Buy Sell Indicator
        for i, row in data.iterrows():
            if i==0:
                data["STX"]="NA"
            elif (data.loc[i,"ST"]<data.loc[i,"close"]) :
                data.loc[i,"STX"]="green"
            else:
                data.loc[i,"STX"]="red"
        return data

In [44]:
def getDataT(token, fromm, to, time_frame):
        global last_data
        data=re.get(f"https://api.kite.trade/instruments/historical/{token}/{time_frame}?from={fromm}&to={to}",headers=headers)
#         print("data-->", data.json())
        newrow=data.json()['data']['candles'][-1]
        return newrow

In [52]:
from dateutil import parser
import requests as re
API_KEY="rfnqn5jyts2ljcq1"
# RUBbPfsVE3sbiBpIk109eAY4L8cOvdnv
# VkjKYp51T2TgvvHNPukdY5K2gRttR3W7
ACCESS_TOKEN="A7WaKJIzhjURGiOD6DVicwSqyIrS7Cak"
headers={"X-Kite-Version":'3','Authorization':"token "+API_KEY+":"+ACCESS_TOKEN}
ROOT_ENDPOINT="https://api.kite.trade"
API_SECRET="4t8tlmijcewpoov8nayd978x7bpj4yif"



interval=3
import datetime,time

from_date = datetime.datetime.now()-datetime.timedelta(days=90)
to_date = datetime.datetime.now()-datetime.timedelta(minutes=interval)
if interval==1:
    time_frame = 'minute'
    
else:
    time_frame = str(interval)+'minute'
    
last_row = getData(5097729,from_date, to_date, time_frame )
print("last_roe",type(last_row))
# last_row = SuperFunc(5097729,from_date, to_date, time_frame )

last_time=last_row["Timestamp"]
# last_signal=last_row[16]
print('running',datetime.datetime.now(),{last_time,last_row["Timestamp"]},{last_row["open"]},{last_row["high"]},{last_row["low"]},{last_row["close"]},{last_row["volume"]},{last_row["ST"]},{last_row["STX"]},'side=sell'if last_row["STX"]=='red' else 'side=buy','qty=1','change=True')
#         adding data in file here
# f = open("marketData.txt", "a")
# d = f"{datetime.datetime.now()},{last_time,last_row["Timestamp"]},{last_row["open"]},{last_row["high"]},{last_row["low"]},{last_row["close"]},{last_row["volume"]},{last_row["ST"]},{last_row["STX"]},{'side=sell'if last_row["STX"]=='red' else 'side=buy'},{'qty=2'},{'change=True' if last_row["STX"]!=last_signal else 'change=False'}"
# f.write(d)
# f.write(f"{datetime.datetime.now()},{last_time,last_row.Timestamp},{last_row.open},{last_row.high},{last_row.low},{last_row.close},{last_row["volume"]},{last_row.ST},{last_row.STX},{'side=sell'if last_row.STX=='red' else 'side=buy'},{'qty=2'},{'change=True' if last_row.STX!=last_signal else 'change=False'}")
# order_plc =  place_order(ROOT_ENDPOINT,headers,"NSE","ICICIBANK",1000,'SELL'if last_row.STX=='red' else 'BUY')
# print(order_plc)

# running 2023-03-24 13:58:41.163117 2023-03-24T13:54:00+0530 3357.65 3359.65 3355.3 3357.8 424 3367.48 red side=sell qty 1or2 change=True
# t1 = threading.Thread(target=getDataThread)
# t1.start()
while True:
    time.sleep(1)
    if str(parser.parse(last_time.replace("T"," "))+datetime.timedelta(minutes=interval*2))<str(datetime.datetime.now()):
        
        from_date = datetime.datetime.now()-datetime.timedelta(days=90)
        to_date = datetime.datetime.now()-datetime.timedelta(minutes=interval)
        last_row = getData(5097729,from_date, to_date, time_frame )
       
        last_time=last_row["Timestamp"]
        print('running',datetime.datetime.now(),{last_time,last_row["Timestamp"]},{last_row["open"]},{last_row["high"]},{last_row["low"]},{last_row["close"]},{last_row["volume"]},{last_row["ST"]},{last_row["STX"]},'side=sell'if last_row["STX"]=='red' else 'side=buy','qty=1','change=True')
#         adding data in file here
#         f = open("marketData.txt", "a")
#         d = f"{datetime.datetime.now()},{last_time,last_row.Timestamp},{last_row.open},{last_row.high},{last_row.low},{last_row.close},{last_row.volume},{last_row.ST},{last_row.STX},{'side=sell'if last_row.STX=='red' else 'side=buy'},{'qty=2'},{'change=True' if last_row.STX!=last_signal else 'change=False'}"
#         f.write(d)
#         print(f"{datetime.datetime.now()},{last_time,last_row.Timestamp},{last_row.open},{last_row.high},{last_row.low},{last_row.close},{last_row.volume},{last_row.ST},{last_row.STX},{'side=sell'if last_row.STX=='red' else 'side=buy'},{'qty=2'},{'change=True' if last_row.STX!=last_signal else 'change=False'}")

#         last_signal=last_row.STX
#         if last_row.STX!=last_signal:
#             print("order called")
#               order_plc =  place_order(ROOT_ENDPOINT,headers,"NSE","ICICIBANK",1000,'SELL'if last_row.STX=='red' else 'BUY')
#             print(order_plc)



<class 'dict'>
last_roe <class 'dict'>
running 2023-04-05 13:28:41.744522 {'2023-04-05T13:24:00+0530'} {3659.3} {3662.35} {3657.55} {3660.6} {1304} {3674.2} {'red'} side=sell qty=1 change=True
<class 'dict'>
running 2023-04-05 13:30:15.031103 {'2023-04-05T13:27:00+0530'} {3659.65} {3661.95} {3659.65} {3661.95} {381} {3674.32} {'red'} side=sell qty=1 change=True
<class 'dict'>
running 2023-04-05 13:33:14.625370 {'2023-04-05T13:30:00+0530'} {3665.9} {3665.9} {3665.05} {3665.9} {372} {3678.96} {'red'} side=sell qty=1 change=True
<class 'dict'>
running 2023-04-05 13:36:16.107097 {'2023-04-05T13:33:00+0530'} {3662.95} {3663.75} {3661.65} {3663.55} {292} {3676.26} {'red'} side=sell qty=1 change=True
<class 'dict'>
running 2023-04-05 13:39:15.387371 {'2023-04-05T13:36:00+0530'} {3659.75} {3662.0} {3659.4} {3660.95} {447} {3674.4} {'red'} side=sell qty=1 change=True


ConnectionError: HTTPSConnectionPool(host='api.kite.trade', port=443): Max retries exceeded with url: /instruments/historical/5097729/3minute?from=2023-01-05%2013:48:06.707168&to=2023-04-05%2013:45:06.707168 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002159B239930>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [51]:
last_time

'2023-04-05T16:42:00+0530'